In [740]:
%%writefile Openmp.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <time.h>

#define N 100000
#define M 10
#define K 128
void bucket_sort_parallel(int arr[], int n);
void insert_into_bucket(int arr[], int start, int end, int buckets[][N], int bucket_counts[], int range);
void sort_buckets(int buckets[][N], int bucket_counts[], int start, int end);

int main() {
    int arr[N];
     clock_t start, end;
    double total_cpu_time_used = 0.0;
 for (int iter = 0; iter < 10; iter++) {
  srand(42);
    for (int i = 0; i < N; i++) {
        arr[i] = rand() % 100;
    }




        start = clock();
        bucket_sort_parallel(arr, N);
        end = clock();

        double cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
        total_cpu_time_used += cpu_time_used;

        // Uncomment below lines to print the sorted array for each iteration
        //for (int i = 0; i < N; i++) {
             //printf("%d ", arr[i]);
         //}
         //printf("\n");

        printf("Iteration %d: Time taken: %f seconds\n", iter+1, cpu_time_used);
    }

    printf("Average running time over 10 iterations: %f seconds\n", total_cpu_time_used / 10);

    return 0;
}

void bucket_sort_parallel(int arr[], int n) {
    int buckets[M][N] = {0};
    int bucket_counts[M] = {0};
    int range = 100 / M;


    #pragma omp parallel num_threads(K)
    {
        int thread_id = omp_get_thread_num();
        int items_per_thread = n / K;
        int start = thread_id * items_per_thread;
        int end = start + items_per_thread - 1;
        if (thread_id == K - 1) end = n - 1;
        insert_into_bucket(arr, start, end, buckets, bucket_counts, range);
    }


    #pragma omp parallel num_threads(K)
    {
        int thread_id = omp_get_thread_num();
        int buckets_per_thread = M / K;
        int start = thread_id * buckets_per_thread;
        int end = start + buckets_per_thread - 1;
        if (thread_id == K - 1) end = M - 1;
        sort_buckets(buckets, bucket_counts, start, end);
    }


    int index = 0;
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < bucket_counts[i]; j++) {
            arr[index++] = buckets[i][j];
        }
    }
}

void insert_into_bucket(int arr[], int start, int end, int buckets[][N], int bucket_counts[], int range) {
    for (int i = start; i <= end; i++) {
        int bucket_index = arr[i] / range;
        #pragma omp critical
        {
            buckets[bucket_index][bucket_counts[bucket_index]++] = arr[i];
        }
    }
}

void sort_buckets(int buckets[][N], int bucket_counts[], int start, int end) {
    for (int i = start; i <= end; i++) {
        if (bucket_counts[i] > 0) {

            for (int j = 1; j < bucket_counts[i]; j++) {
                int key = buckets[i][j];
                int k = j - 1;
                while (k >= 0 && buckets[i][k] > key) {
                    buckets[i][k + 1] = buckets[i][k];
                    k = k - 1;
                }
                buckets[i][k + 1] = key;
            }
        }
    }
}


Overwriting Openmp.c


In [741]:
!gcc -fopenmp Openmp.c -o Openmp

In [742]:
!./Openmp



Iteration 1: Time taken: 0.763939 seconds
Iteration 2: Time taken: 0.779748 seconds
Iteration 3: Time taken: 0.771245 seconds
Iteration 4: Time taken: 0.773546 seconds
Iteration 5: Time taken: 0.995352 seconds
Iteration 6: Time taken: 1.251320 seconds
Iteration 7: Time taken: 0.924430 seconds
Iteration 8: Time taken: 0.755531 seconds
Iteration 9: Time taken: 0.760387 seconds
Iteration 10: Time taken: 0.757021 seconds
Average running time over 10 iterations: 0.853252 seconds


In [743]:
%%writefile serial.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define N 100000
#define M 10

void bucket_sort_serial(int arr[], int n);
void insert_into_bucket(int arr[], int start, int end, int buckets[][N], int bucket_counts[], int range);
void sort_buckets(int buckets[][N], int bucket_counts[], int start, int end);

int main() {
    int arr[N];
  clock_t start, end;
    double total_cpu_time_used = 0.0;
 for (int iter = 0; iter < 10; iter++) {
   srand(42);
    for (int i = 0; i < N; i++) {
        arr[i] = rand() % 100; // Example: elements range from 0 to 99
    }


    start = clock();
    bucket_sort_serial(arr, N);
    end = clock();

    double cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
        total_cpu_time_used += cpu_time_used;
    // Print sorted array
    //for (int i = 0; i < N; i++) {
        //printf("%d ", arr[i]);
    //}
    //printf("\n");
 printf("Iteration %d: Time taken: %f seconds\n", iter+1, cpu_time_used);
    }

    printf("Average running time over 10 iterations: %f seconds\n", total_cpu_time_used / 10);

    return 0;
}

void bucket_sort_serial(int arr[], int n) {
    int buckets[M][N] = {0};
    int bucket_counts[M] = {0};
    int range = 100 / M;

    // Distribute elements into buckets
    insert_into_bucket(arr, 0, n - 1, buckets, bucket_counts, range);

    // Sort each bucket
    sort_buckets(buckets, bucket_counts, 0, M - 1);

    // Merge buckets back to original array
    int index = 0;
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < bucket_counts[i]; j++) {
            arr[index++] = buckets[i][j];
        }
    }
}

void insert_into_bucket(int arr[], int start, int end, int buckets[][N], int bucket_counts[], int range) {
    for (int i = start; i <= end; i++) {
        int bucket_index = arr[i] / range;
        buckets[bucket_index][bucket_counts[bucket_index]++] = arr[i];
    }
}

void sort_buckets(int buckets[][N], int bucket_counts[], int start, int end) {
    for (int i = start; i <= end; i++) {
        if (bucket_counts[i] > 0) {
            // Simple insertion sort for demonstration
            for (int j = 1; j < bucket_counts[i]; j++) {
                int key = buckets[i][j];
                int k = j - 1;
                while (k >= 0 && buckets[i][k] > key) {
                    buckets[i][k + 1] = buckets[i][k];
                    k = k - 1;
                }
                buckets[i][k + 1] = key;
            }
        }
    }
}


Overwriting serial.c


In [744]:
!gcc serial.c -o serial

In [745]:
!./serial

Iteration 1: Time taken: 0.764029 seconds
Iteration 2: Time taken: 0.749883 seconds
Iteration 3: Time taken: 0.941045 seconds
Iteration 4: Time taken: 1.222779 seconds
Iteration 5: Time taken: 0.999021 seconds
Iteration 6: Time taken: 0.771160 seconds
Iteration 7: Time taken: 0.758965 seconds
Iteration 8: Time taken: 0.758664 seconds
Iteration 9: Time taken: 0.755355 seconds
Iteration 10: Time taken: 0.747728 seconds
Average running time over 10 iterations: 0.846863 seconds


In [776]:
%%writefile pthreads.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <time.h>

#define N 100000
#define M 10
#define K 128
#define RUNS 10 // Number of runs

int arr[N];
int buckets[M][N] = {0};
int bucket_counts[M] = {0};
int range = 100 / M;
pthread_mutex_t lock[M];

typedef struct {
    int start;
    int end;
    int thread_id;
} thread_data;

void* insert_into_bucket(void* arg);
void* sort_buckets(void* arg);

int main() {
    double total_time_used = 0.0; // For calculating the average time

    for (int run = 0; run < RUNS; run++) { // Outer loop for multiple runs
        srand(42 + run); // Vary the seed slightly for each run
        for (int i = 0; i < N; i++) {
            arr[i] = rand() % 100;
        }

        // Reset bucket counts for each run
        for (int i = 0; i < M; i++) {
            bucket_counts[i] = 0;
        }

        pthread_t threads[K];
        thread_data t_data[K];
        pthread_attr_t attr;
        pthread_attr_init(&attr);

        clock_t start, end;

        start = clock();

        // Initialize mutexes
        for (int i = 0; i < M; i++) {
            pthread_mutex_init(&lock[i], NULL);
        }

        // Creating threads for inserting into buckets
        for (int i = 0; i < K; i++) {
            t_data[i].start = i * (N / K);
            t_data[i].end = (i + 1) * (N / K) - 1;
            if (i == K - 1) t_data[i].end = N - 1; // Last thread may take more
            t_data[i].thread_id = i;

            pthread_create(&threads[i], &attr, insert_into_bucket, (void*)&t_data[i]);
        }

        // Joining threads after insertion
        for (int i = 0; i < K; i++) {
            pthread_join(threads[i], NULL);
        }

        // Creating threads for sorting buckets
        for (int i = 0; i < K; i++) {
            t_data[i].start = i * (M / K);
            t_data[i].end = (i + 1) * (M / K) - 1;
            if (i == K - 1) t_data[i].end = M - 1; // Last thread may take more

            pthread_create(&threads[i], &attr, sort_buckets, (void*)&t_data[i]);
        }

        // Joining threads after sorting
        for (int i = 0; i < K; i++) {
            pthread_join(threads[i], NULL);
        }

        // Merge sorted buckets
        int index = 0;
        for (int i = 0; i < M; i++) {
            for (int j = 0; j < bucket_counts[i]; j++) {
                arr[index++] = buckets[i][j];
            }
        }

        end = clock();

        double cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
        total_time_used += cpu_time_used;

        // Destroy mutexes
        for (int i = 0; i < M; i++) {
            pthread_mutex_destroy(&lock[i]);
        }
    }

    double average_time = total_time_used / RUNS;
    printf("Average time taken: %f seconds\n", average_time);

    return 0;
}

void* insert_into_bucket(void* arg) {
    thread_data* t_data = (thread_data*)arg;
    int start = t_data->start;
    int end = t_data->end;

    for (int i = start; i <= end; i++) {
        int bucket_index = arr[i] / range;
        pthread_mutex_lock(&lock[bucket_index]);
        buckets[bucket_index][bucket_counts[bucket_index]++] = arr[i];
        pthread_mutex_unlock(&lock[bucket_index]);
    }

    pthread_exit(NULL);
}

void* sort_buckets(void* arg) {
    thread_data* t_data = (thread_data*)arg;
    int start = t_data->start;
    int end = t_data->end;

    for (int i = start; i <= end; i++) {
        if (bucket_counts[i] > 0) {
            // Insertion sort
            for (int j = 1; j < bucket_counts[i]; j++) {
                int key = buckets[i][j];
                int k = j - 1;
                while (k >= 0 && buckets[i][k] > key) {
                    buckets[i][k + 1] = buckets[i][k];
                    k = k - 1;
                }
                buckets[i][k + 1] = key;
            }
        }
    }

    pthread_exit(NULL);
}


Overwriting pthreads.c


In [777]:
!gcc pthreads.c -o threadprogram -lpthread

In [778]:
!./threadprogram

Average time taken: 0.924749 seconds
